### Import and clean data

In [ ]:
""" 
Load and read the 'imbd.xlsx' file. Read the 'imbd' sheet into a DataFrame named df.

- What are the dimensions of the DataFrame?
- What are the column titles and the types of data in each?
- Examine the first 10 rows of data.
"""

import pandas as pd

xls = pd.ExcelFile('imdb.xlsx')
df = xls.parse('imdb')
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head(10)

In [ ]:
"""
Import the "directors" and "countries" sheets into their own DataFrames.

Check the "Directors" sheet for duplicates based on the "id" column.
Remove duplicates.
"""

df_directors = xls.parse('directors')
df_directors["id"].value_counts()

In [ ]:
df_directors = df_directors.drop_duplicates()
df_directors.shape

In [ ]:
df_directors["id"].value_counts()

In [ ]:
df_countries = xls.parse('countries')
df_countries.shape

In [ ]:
""" 
Join the data with an "inner join".
Do this by merging into df.
"""

df = pd.merge(left=df, right=df_countries, 
              how='inner', left_on='country_id', 
              right_on='id')

In [ ]:
""" 
Join the data with an "inner join".
Do this by merging into df.
"""

df = pd.merge(left=df, right=df_directors, 
              how='inner', left_on='director_id', 
              right_on='id')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
""" 
Print the first ten rows of movie titles. 
There's an extra character. 
Remove it from the data using str.replace.
"""

df["movie_title"][:10]

In [ ]:
df["movie_title"] = df["movie_title"].str.replace('Ê','')
df["movie_title"][:10]

In [ ]:
"""
Get the summary statistics for imdb_score and gross.
"""

score_gross = ["imdb_score", "gross"]
df[score_gross].describe()

### Import and store in database

In [ ]:
#import for connecting to Azure database
import pyodbc

#connect to Azure database
#insert server info and credentials for your database

server = '<INSERT SERVER NAME>'
database = '<INSERT DATABASE NAME>'
username = '<INSERT USERNAME>'
password = "<INSERT PASSWORD>"

driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

#create cursor for executing queries
cursor = cnxn.cursor()

In [ ]:
#define function to iterate over db cursor and print each record
def cursor_fetch(cursor):
    row = cursor.fetchone()
    while row:
        print (row)
        row = cursor.fetchone()

In [ ]:
#get all available tables in database
cursor.execute('''SELECT * FROM INFORMATION_SCHEMA.TABLES;''')
cursor_fetch(cursor)

In [ ]:
#create imdb table
cursor.execute('''CREATE TABLE imdb (
    [movie_title] VARCHAR(200),
    [director_id] INT,
    [country_id] INT,
    [content_rating] VARCHAR(200),
    [title_year] INT,
    [imdb_score] FLOAT,
    [gross] INT,
    [duration] INT,
    [country] VARCHAR(200),
    [director_name] VARCHAR(200)
);''')

In [ ]:
#query new table
cursor.execute('SELECT * FROM imdb;')
cursor_fetch(cursor)

In [ ]:
#get first 5 rows of data in dataframe
df_head = df.head()
df_head

In [ ]:
#insert first 5 rows of data into new table as test
for index, row in df_head.iterrows():
    cursor.execute("INSERT INTO imdb VALUES (?,?,?,?,?,?,?,?,?,?);", 
        row['movie_title'], 
        row['director_id'], 
        row['country_id'],
        row['content_rating'],
        row['title_year'],
        row['imdb_score'],
        row['gross'],
        row['duration'],
        row['country'],
        row['director_name']
    )

In [ ]:
#get count of rows
cursor.execute('''SELECT COUNT(*) cnt from imdb;''')
cursor_fetch(cursor)

In [ ]:
#get entire rows
cursor.execute('''SELECT * from imdb;''')
cursor_fetch(cursor)

In [ ]:
#get specific columns
cursor.execute('''SELECT movie_title from imdb;''')
cursor_fetch(cursor)

In [ ]:
#get specific columns and filter rows
cursor.execute('''SELECT movie_title, title_year from imdb WHERE director_name = 'Francis Ford Coppola';''')
cursor_fetch(cursor)

In [ ]:
#get specific columns and filter rows
cursor.execute('''SELECT movie_title, gross from imdb WHERE gross > 78000000;''')
cursor_fetch(cursor)

In [ ]:
#get top 2 rows from table and store as dataframe
#using pandas read_sql method
df1 = pd.read_sql('''SELECT TOP 2 * FROM imdb;''', cnxn)
df1

In [ ]:
#empty sql table
cursor.execute("DELETE FROM imdb;") 

In [ ]:
#insert all rows of data into new table 
for index, row in df.iterrows():
    cursor.execute("INSERT INTO imdb VALUES (?,?,?,?,?,?,?,?,?,?);", 
        row['movie_title'], 
        row['director_id'], 
        row['country_id'],
        row['content_rating'],
        row['title_year'],
        row['imdb_score'],
        row['gross'],
        row['duration'],
        row['country'],
        row['director_name']
    )

In [ ]:
#get top 50 earning movies (based on gross) and store as dataframe
df1 = pd.read_sql('''SELECT TOP 50 * FROM imdb ORDER BY gross DESC;''', cnxn)
df1

In [ ]:
#create new table
cursor.execute('''CREATE TABLE imdb_top50 (
    [movie_title] VARCHAR(200),
    [director_id] INT,
    [country_id] INT,
    [content_rating] VARCHAR(200),
    [title_year] INT,
    [imdb_score] FLOAT,
    [gross] INT,
    [duration] INT,
    [country] VARCHAR(200),
    [director_name] VARCHAR(200)
)''')

In [ ]:
#insert entire dataset into table
for index, row in df1.iterrows():
    cursor.execute("INSERT INTO imdb_top50 VALUES (?,?,?,?,?,?,?,?,?,?)", 
        row['movie_title'], 
        row['director_id'], 
        row['country_id'],
        row['content_rating'],
        row['title_year'],
        row['imdb_score'],
        row['gross'],
        row['duration'],
        row['country'],
        row['director_name']
    )

In [ ]:
#get all movies
df2 = pd.read_sql('''SELECT * FROM imdb;''', cnxn)
df2

### Data visualization

In [ ]:
"""
Make a simple scatter plot comparing gross to imdb_score for movies released in or after 2000.
It may be useful to scale the axis demarking gross.
"""

#get movies released in or after 2000
df_2000 = df2[df2['title_year'] >= 2000]
df_2000

In [ ]:
#create series with movie imdb score
x1 = df_2000['imdb_score']
x1

In [ ]:
#you'll see the "gross" column is in millions
df_2000['gross'].head()

In [ ]:
#create series with gross value (in millions)
x2 = df_2000['gross'] / (10**6)
x2

In [ ]:
import matplotlib.pyplot as plt

%pylab inline

In [ ]:
#create scatter plot for x1 and x2
plt.scatter(x2, x1, marker='o', color='b', alpha=0.7,
           s=50)

plt.xlabel('Gross (Millions)')
plt.ylabel('Rating')
plt.legend(loc='best')

plt.show()

### Other data exploration

In [ ]:
""" 
Who is the director with the most movies?
"""

#value_counts() automatically sorts in desc order
#take the first record
df2["director_name"].value_counts()[0:1]

In [ ]:
"""
List all of this director's movies and their ratings.
"""

nolan = df2["director_name"] == "Christopher Nolan"
df[nolan][["movie_title","imdb_score"]]

In [ ]:
"""
What is this director's average rating? 
"""

df2[nolan]["imdb_score"].mean()

### Data visualization

In [ ]:
"""
Using numpy and pyplot, make an overlapping histogram that compares the imbd scores between R-Rated movies and PG-13 movies.
Describe your plot. 
"""

df_R = df2[df2["content_rating"]=='R']
df_PG13 = df2[df2["content_rating"]=='PG-13']

series_R = df_R["imdb_score"]
series_PG13 = df_PG13["imdb_score"]

series_PG13

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.hist([series_R, series_PG13], alpha=0.7, color=['red', 'blue'], label=['R-Rated Movies', 'PG-13 Movies'], bins='auto')

plt.xlabel('IMDB Score')
plt.ylabel('Count')

plt.legend(loc='best')
plt.title("Score Distribution of R-Rated & PG-13 Movies")

plt.show() 

### Close database connection

In [ ]:
#close cursor and db connection
cursor.close()
cnxn.close()